In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sys.path.insert(0, "../scripts/")
from spliting import SplitData


In [7]:
df =pd.read_csv('../data/cleaned_data.csv')
df.sample(10)

,auction_id,experiment,date,hour,device_make,platform_os,browser,user_response
902,81917d68-d672-42bd-83dd-a96dba48a64a,exposed,2020-07-09,18,Generic Smartphone,6,Chrome Mobile,0
41,13e225de-5cb9-4d9b-96fb-aa137c643ce3,control,2020-07-03,15,Samsung SM-G930F,6,Facebook,1
1166,e58ffe4a-df41-4b5a-a309-9b8d0ce64976,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0
932,8c6bd9c8-b63b-4bfa-a047-023502992d3e,exposed,2020-07-05,7,Generic Smartphone,6,Chrome Mobile,0
410,bdf43d25-fd7d-482e-a383-46b217cede7d,exposed,2020-07-06,9,Generic Smartphone,6,Chrome Mobile,1
1030,b021ba43-576a-4c85-972d-fd52351a77ef,exposed,2020-07-06,10,Samsung SM-G950F,6,Chrome Mobile WebView,0
309,99740d86-02c1-4cb4-aeca-827cd6f70cfb,control,2020-07-04,22,LG-$2,6,Facebook,1
730,4441b628-9390-41fb-b769-359a27510fbf,control,2020-07-09,14,Generic Smartphone,6,Chrome Mobile,0
918,87fbdaef-9c71-4fa6-a675-e90b6c6833d0,exposed,2020-07-08,3,Generic Smartphone,6,Chrome Mobile,0
1083,c3ba13d2-445c-4860-bc56-9b60b27e4698,control,2020-07-09,17,Generic Smartphone,6,Chrome Mobile,0


In [10]:
browser_df=df.drop(['platform_os'],axis=1)
browser_df.head(10)


,auction_id,experiment,date,hour,device_make,browser,user_response
0,008aafdf-deef-4482-8fec-d98e3da054da,exposed,2020-07-04,16,Generic Smartphone,Chrome Mobile,1
1,00b6fadb-10bd-49e3-a778-290da82f7a8d,control,2020-07-08,4,Samsung SM-A202F,Facebook,1
2,018af862-486e-4da1-a85b-71872120e57c,control,2020-07-03,15,Generic Smartphone,Chrome Mobile,1
3,023ec7b5-cb8f-49a5-995f-e0d7c2f702e5,exposed,2020-07-09,13,Samsung SM-G935F,Facebook,1
4,02efdb70-8596-4f3f-b0b2-b91e194f61f7,exposed,2020-07-05,6,Generic Smartphone,Chrome Mobile,1
5,0332a7f7-0336-4d48-96ad-75da00d916c0,control,2020-07-04,16,Generic Smartphone,Chrome Mobile,1
6,03948b84-b5b1-4f26-81fc-e953218aef1a,control,2020-07-09,20,Samsung SM-G930F,Facebook,1
7,03ee1780-1fa3-4bec-926c-d543a81a6653,exposed,2020-07-04,11,Samsung SM-G981B,Chrome Mobile WebView,1
8,046644f4-b30c-48f9-b069-f804a97b77d9,exposed,2020-07-05,2,Generic Smartphone,Chrome Mobile WebView,1
9,04b69bd8-4849-4489-adcf-08b96a265c40,control,2020-07-07,15,Generic Smartphone,Chrome Mobile,1


In [11]:
browser_df.to_csv('../data/browser_clean_data.csv')

In [12]:
platform_df=df.drop(['browser'],axis=1)
platform_df.sample(10)

,auction_id,experiment,date,hour,device_make,platform_os,user_response
23,0aaf0e53-0aef-48e1-86b8-77a751b6ad2b,control,2020-07-08,14,Generic Smartphone,6,1
979,9b7a2497-f2f3-4cfd-bf1b-bf533fa39f4f,exposed,2020-07-10,2,Generic Smartphone,6,0
416,c03a506f-92e4-41e1-8859-841d21f18349,exposed,2020-07-04,6,Samsung SM-G965F,6,1
789,577d8982-42dc-401d-8fc3-56e394a96782,exposed,2020-07-09,16,Generic Smartphone,6,0
889,7d6e7872-c491-4fa5-9436-87e13c3e32dc,exposed,2020-07-08,21,Samsung SM-G950F,6,0
754,4e013f88-ee69-49b8-955d-c661d491a450,exposed,2020-07-07,17,Samsung SM-G980F,6,0
369,ad5c63c0-72b0-404b-8a50-501f8246be93,exposed,2020-07-09,11,Generic Smartphone,6,1
987,9ea96c35-6b58-477a-bcca-392be377e13b,control,2020-07-10,3,VFD 820,6,0
350,a6d7e9ac-1bf2-4ae3-b7c5-92b26ab22c27,exposed,2020-07-04,13,F3311,6,1
494,e21bd15a-bd75-44d7-accc-20b3580f6055,control,2020-07-04,16,Generic Smartphone,6,1


In [13]:
platform_df.to_csv('../data/platform_clean_data.csv')

# Accessing DVC version

In [20]:
import dvc.api

<h3>Accessing Browser Data</h3>

In [33]:
with dvc.api.open(
    'data/browser_clean_data.csv',
    repo="https://github.com/abu-bakarr/10_Academy_Week2_Group",
    
) as f:
    browser_df = pd.read_csv(f)
    
browser_df.drop(['auction_id','Unnamed: 0'],axis=1,inplace=True)


In [34]:
with dvc.api.open(
    "data/platform_clean_data.csv",
    repo="https://github.com/abu-bakarr/10_Academy_Week2_Group",
) as fd:
    platform_df = pd.read_csv(fd)
    
platform_df.drop(["auction_id", "Unnamed: 0"], axis=1, inplace=True)


In [35]:
browser_df.sample(3)

,experiment,date,hour,device_make,browser,user_response
1114,control,2020-07-03,15,Generic Smartphone,Chrome Mobile,0
1098,control,2020-07-05,4,Generic Smartphone,Chrome Mobile,0
681,control,2020-07-09,18,Generic Smartphone,Chrome Mobile,0


In [36]:
platform_df.sample(3)

,experiment,date,hour,device_make,platform_os,user_response
1020,exposed,2020-07-07,9,Generic Smartphone,6,0
1184,exposed,2020-07-03,1,Generic Smartphone,6,0
765,control,2020-07-10,5,Samsung SM-A202F,6,0


<h5>Setting up training,validation and testing data</h5>

In [38]:
split_browser_data=SplitData(browser_df)

In [39]:
split_platform_data=SplitData(platform_df)

In [41]:
browser_data=split_browser_data.test_validate_train(0.7,0.2,'user_response')

(870, 5)
(870,)
(248, 5)
(248,)
(125, 5)
(125,)


In [43]:
browser_x_train=browser_data['x_train']
browser_y_train=browser_data['y_train']
browser_x_valid=browser_data['x_valid']
browser_y_valid=browser_data['y_valid']
browser_x_test=browser_data['x_test']
browser_y_test=browser_data['y_test']

In [46]:
browser_x_valid.head(3)

,experiment,date,hour,device_make,browser
30,exposed,2020-07-08,16,Generic Smartphone,Chrome Mobile
29,control,2020-07-03,15,Samsung SM-G950F,Chrome Mobile WebView
28,exposed,2020-07-05,3,Generic Smartphone,Chrome Mobile


In [47]:
platform_data = split_platform_data.test_validate_train(0.7, 0.2, "user_response")


(870, 5)
(870,)
(248, 5)
(248,)
(125, 5)
(125,)


In [48]:
platform_x_train =platform_data["x_train"]
platform_y_train =platform_data["y_train"]
platform_x_valid =platform_data["x_valid"]
platform_y_valid =platform_data["y_valid"]
platform_x_test = platform_data["x_test"]
platform_y_test = platform_data["y_test"]

In [49]:
platform_x_train.head(3)

,experiment,date,hour,device_make,platform_os
621,exposed,2020-07-03,0,Generic Smartphone,6
832,control,2020-07-03,15,Generic Smartphone,6
831,exposed,2020-07-04,19,Generic Smartphone,6
